<a href="https://colab.research.google.com/github/Brahm13/Zoomcamp2024/blob/main/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [9]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
index = 1
total = 0
for sqrt_value in generator:
  print(f"{index}. = {sqrt_value} = {total+sqrt_value}")
  total += sqrt_value
  index += 1


# print(total)


1. = 1.0 = 1.0
2. = 1.4142135623730951 = 2.414213562373095
3. = 1.7320508075688772 = 4.146264369941973
4. = 2.0 = 6.146264369941973
5. = 2.23606797749979 = 8.382332347441762
6. = 2.449489742783178 = 10.83182209022494
7. = 2.6457513110645907 = 13.47757340128953
8. = 2.8284271247461903 = 16.30600052603572
9. = 3.0 = 19.30600052603572
10. = 3.1622776601683795 = 22.4682781862041
11. = 3.3166247903554 = 25.7849029765595
12. = 3.4641016151377544 = 29.249004591697254
13. = 3.605551275463989 = 32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [34]:
import dlt
import duckdb

pipeline_var = dlt.pipeline(pipeline_name="one_time_load",destination="duckdb", dataset_name="human")

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


# for person in people_1():
#   print(f"{person}")


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


print("Appending the two table....")
load_to_table1 = pipeline_var.run(people_1(),table_name="age",write_disposition="replace")

conn = duckdb.connect(f"{pipeline_var.pipeline_name}.duckdb")

conn.sql(f"Set Search_path = {pipeline_var.dataset_name}")
print("First table age")
display(conn.sql("select * from age"))

load_to_table2 = pipeline_var.run(people_2(),table_name="age",write_disposition="append")
print("Sum of two table with append")
display(conn.sql("select sum(age) as append_grand_total from age"))
display(conn.sql("select * from age"))

print("merging and summing age of two table")
pipeline_var.run(people_1(),table_name="age2",write_disposition="merge")
pipeline_var.run(people_2(),table_name="age2",write_disposition="merge", primary_key="id")
print("Sum of two table with merge")
display(conn.sql("select sum(age) as merge_grand_total from age2"))
display(conn.sql("select * from age2"))


{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
Appending the two table....
First table age


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708187366.3004792 │ BCmczFTH9VfhUQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708187366.3004792 │ o1S/WofpoKOiGQ │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708187366.3004792 │ CFAxDgld6rg7tQ │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708187366.3004792 │ O2mCk0ZG/oRfEQ │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1708187366.3004792 │ 3OBJS6TpI9FwfA │ NULL       │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┴────────────┘

Sum of two table with append


┌────────────────────┐
│ append_grand_total │
│       int128       │
├────────────────────┤
│                353 │
└────────────────────┘

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708187366.3004792 │ BCmczFTH9VfhUQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708187366.3004792 │ o1S/WofpoKOiGQ │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708187366.3004792 │ CFAxDgld6rg7tQ │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708187366.3004792 │ O2mCk0ZG/oRfEQ │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1708187366.3004792 │ 3OBJS6TpI9FwfA │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1708187366.9599411 │ Up1uw0NoLfyPIw │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1708187366.9599411 │ nUVjYHtcmLktGg │ Job_4      │
│     5 │ 

merging and summing age of two table
Sum of two table with merge


┌───────────────────┐
│ merge_grand_total │
│      int128       │
├───────────────────┤
│               266 │
└───────────────────┘

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708187367.604145  │ Vy8CSVNZWLxo5g │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708187367.604145  │ vgSo1CEzgCv3Cg │ NULL       │
│     5 │ Person_5 │    35 │ City_B  │ 1708187368.3989735 │ C5sey+ABtcJq3g │ Job_5      │
│     8 │ Person_8 │    38 │ City_B  │ 1708187368.3989735 │ 5ZDc8ZejAc155w │ Job_8      │
│     7 │ Person_7 │    37 │ City_B  │ 1708187368.3989735 │ OLO0oDvBMHwodw │ Job_7      │
│     4 │ Person_4 │    34 │ City_B  │ 1708187368.3989735 │ pEJfdr5DHvGlIA │ Job_4      │
│     3 │ Person_3 │    33 │ City_B  │ 1708187368.3989735 │ qsHVsvKDnm1EkA │ Job_3      │
│     6 │ 

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [1]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX